In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib as mlt

In [7]:
df = pd.read_csv('marathon_raw.csv', sep=';')
df.head(1)

,Name_country,Age_group,Number,Time,Pace_0_5,Pace_35_40,First_half,Second_half
0,"Adola, Guye (ETH)",M30,2.0,02:05:48,02:53,03:12,01:00:48,01:05:00


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Name_country  1 non-null      object 
 1   Age_group     1 non-null      object 
 2   Number        1 non-null      float64
 3   Time          1 non-null      object 
 4   Pace_0_5      1 non-null      object 
 5   Pace_35_40    1 non-null      object 
 6   First_half    1 non-null      object 
 7   Second_half   1 non-null      object 
dtypes: float64(1), object(7)
memory usage: 1.8+ KB


In [12]:
df['Country'] = df['Name_country'].str[-4:-1]
df.head(1)

,Name_country,Age_group,Number,Time,Pace_0_5,Pace_35_40,First_half,Second_half,Country
0,"Adola, Guye (ETH)",M30,2.0,02:05:48,02:53,03:12,01:00:48,01:05:00,ETH


In [28]:
df['Gender'] = df['Age_group'].str[0:1]
df.head(1)

,Name_country,Age_group,Number,Time,Pace_0_5,Pace_35_40,First_half,Second_half,Country,Gender
0,"Adola, Guye (ETH)",M30,2.0,02:05:48,02:53,03:12,01:00:48,01:05:00,ETH,M


In [29]:
df['Age'] = df['Age_group'].str[1:]
df.head(1)

,Name_country,Age_group,Number,Time,Pace_0_5,Pace_35_40,First_half,Second_half,Country,Gender,Age
0,"Adola, Guye (ETH)",M30,2.0,02:05:48,02:53,03:12,01:00:48,01:05:00,ETH,M,30


In [30]:
df2 = pd.read_csv('wikipedia-iso-country-codes.csv')
df2.head(2)

,English short name lower case,Alpha-2 code,Alpha-3 code,Numeric code,ISO 3166-2
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX


In [31]:
df2.rename(columns={'Alpha-3 code': 'Country', 'English short name lower case': 'Country_long'}, inplace=True)
df2

,Country_long,Alpha-2 code,Country,Numeric code,ISO 3166-2
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX
2,Albania,AL,ALB,8,ISO 3166-2:AL
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ
4,American Samoa,AS,ASM,16,ISO 3166-2:AS
...,...,...,...,...,...
241,Wallis and Futuna,WF,WLF,876,ISO 3166-2:WF
242,Western Sahara,EH,ESH,732,ISO 3166-2:EH
243,Yemen,YE,YEM,887,ISO 3166-2:YE
244,Zambia,ZM,ZMB,894,ISO 3166-2:ZM


In [32]:
df2.loc[:, ['Country_long','Country']].head()

,Country_long,Country
0,Afghanistan,AFG
1,Åland Islands,ALA
2,Albania,ALB
3,Algeria,DZA
4,American Samoa,ASM


In [33]:
df_merged = pd.merge(df, 
                      df2.loc[:, ['Country_long','Country']], 
                      on ='Country', 
                      how ='inner')

In [34]:
df_merged

,Name_country,Age_group,Number,Time,Pace_0_5,Pace_35_40,First_half,Second_half,Country,Gender,Age,Country_long
0,"Adola, Guye (ETH)",M30,2.0,02:05:48,02:53,03:12,01:00:48,01:05:00,ETH,M,30,Ethiopia
